In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
device

device(type='cuda', index=0)

In [21]:
dataset = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/jfk.csv')

In [22]:
dataset.shape

(8757, 1049)

In [23]:
dataset.head(3)

,Date,Hour,1,10,100,101,102,106,107,108,...,91_lag_3,92_lag_3,93_lag_3,94_lag_3,95_lag_3,96_lag_3,97_lag_3,98_lag_3,99_lag_3,arrival_lag_3
0,2018-01-01,3,0,0,0,0,0,0,0,0,...,1.0,1.0,0.0,1.0,6.0,0.0,1.0,0.0,0.0,6.0
1,2018-01-01,4,0,3,0,0,1,0,0,1,...,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,6.0
2,2018-01-01,5,0,4,0,0,1,2,3,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0


In [24]:
lag_columns = [c for c in dataset.columns if 'lag' in c]
len(lag_columns)

777

In [25]:
dataset = dataset[[c for c in dataset.columns if c not in lag_columns]]
dataset.shape

(8757, 272)

In [26]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in dataset.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']
len(targetColumns)

258

In [27]:
features_cols = [c for c in dataset.columns if c not in targetColumns and c not in DateColumns]
len(features_cols)

13

In [28]:
sep = int(0.75*len(dataset))
print(sep)


trainData = dataset[:sep]
testData = dataset[sep:]

print(trainData.shape)
print(testData.shape)

6567
(6567, 272)
(2190, 272)


In [29]:
X_train = trainData[features_cols].values
X_train = torch.tensor(X_train).float().to(device)
print(X_train.shape)

y_train = trainData[targetColumns].values
y_train = torch.tensor(y_train).float().to(device)
print(y_train.shape)

X_test = testData[features_cols].values
X_test = torch.tensor(X_test).float().to(device)
print(X_test.shape)

y_test = testData[targetColumns].values
y_test = torch.tensor(y_test).float().to(device)
print(y_test.shape)

torch.Size([6567, 13])
torch.Size([6567, 258])
torch.Size([2190, 13])
torch.Size([2190, 258])


In [30]:
def create_inout_sequences(x,y, tw):
    inout_seq = []
    L = len(x)
    for i in range(L-tw):
        train_seq_x = x[i:i+tw]
        train_seq_y = y[i:i+tw]
        train_seq = torch.cat((train_seq_x,train_seq_y),axis=1)
        
#         train_label = y[i+tw:i+tw+1]
        train_label = y[i+1:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [ ]:
bptt=6

In [31]:
train_inout_seq = create_inout_sequences(X_train,y_train, bptt)

In [32]:
train_inout_seq[0][0].shape,train_inout_seq[0][1].shape

(torch.Size([6, 271]), torch.Size([6, 258]))

In [33]:
class GRU(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1, layers=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.GRU(input_size=input_size, hidden_size=hidden_layer_size, num_layers=layers)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = torch.zeros(layers,1,self.hidden_layer_size)

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
#         return predictions[-1]
        return predictions

In [34]:
layers = 1
model = GRU(input_size = 271, hidden_layer_size=100, output_size=258, layers=layers).to(device)
loss_function = nn.L1Loss()   
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [35]:
epochs = 150

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = torch.zeros(layers, 1, model.hidden_layer_size).to(device)

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 1.42956817
epoch:  26 loss: 1.43398213
epoch:  51 loss: 1.42090619
epoch:  76 loss: 1.37159884
epoch: 101 loss: 1.44581842
epoch: 126 loss: 1.39109576
epoch: 149 loss: 1.3808450699


In [36]:
test_inout_seq = create_inout_sequences(X_test,y_test, bptt)

In [37]:
model.eval()
prediction = []
with torch.no_grad():
    for seq, labels in test_inout_seq:
        model.hidden = torch.zeros(layers, 1, model.hidden_layer_size)
        prediction.append(model(seq)[-1])

In [38]:
y_test_ = torch.stack([labels[-1] for seq, labels in test_inout_seq], axis=0).detach().cpu().numpy()
y_pred_ = torch.stack(prediction).detach().cpu().numpy()

In [39]:
y_test_.shape, y_pred_.shape

((2184, 258), (2184, 258))

In [40]:
r2_score(y_test_, y_pred_, multioutput='variance_weighted')

-0.01414043409617363

In [ ]:
# 300 epoch MSE bptt 12 hidden=100
#0.4597512505107641